In [35]:
import random
from random import choices
import pandas as pd
import numpy as np

In [36]:
# Hearts, Diamonds, Clubs, Spades
suits = ('H','D','S','C')

# Possible Card Ranks
ranking = ('A','2','3','4','5','6','7','8','9','10','J','Q','K')

# Point Val Dict (Dual existence of Ace is defined later)
#card_val = {'A':1  '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, '10':10, 'J':10, 'Q':10, 'K':10, }
# A = 1,11

cards_df = pd.DataFrame({"cards_value": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], "dealer": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]})



In [37]:
cards_df

,cards_value,dealer
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
5,6,6
6,7,7
7,8,8
8,9,9
9,10,10


In [56]:
X= cards_df["cards_value"].reshape()
#y= cards_df["dealer"].values()

AttributeError: 'Series' object has no attribute 'reshape'

In [45]:
# empty dataframe to append results 
game_df = pd.DataFrame({"player" : [], "Strategy":[]})
game_df

,player,Strategy


In [51]:
# Initialize the game Blackjack

win_game = 21
#player = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
start_game = choices(cards_df["cards_value"], k=2)

for player in game_df:
    if start_game == 21:
        game_df.loc[ "Strategy"] = "Stand"

    elif start_game  > 17:
       game_df.loc[ "Strategy"] = "Stand"

    elif start_game < 16:
        game_df.loc[ "Strategy"] = "Hit"

print(game_df)

TypeError: '>' not supported between instances of 'list' and 'int'

In [46]:

start_game = choices(cards_df["cards_value"], k=2)
print(start_game)

[4, 4]


In [52]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

In [53]:
# Scale the data
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

ValueError: Expected 2D array, got 1D array instead:
array=[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.